In [1]:
%reload_ext autoreload
%autoreload 2
#%reload_ext notexbook'
#%texify

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch

import os

os.chdir('/home/axtr7550/Chromosome_project/cellbgnet/')
import cellbgnet
import cellbgnet.utils

from cellbgnet.datasets import DataSimulator
from cellbgnet.utils.hardware import cpu, gpu
from cellbgnet.model import CellBGModel
from cellbgnet.simulation.psf_kernel import SMAPSplineCoefficient
from cellbgnet.generic.emitter import EmitterSet
from cellbgnet.train_loss_infer import generate_probmap_cells

os.chdir('/home/axtr7550/Chromosome_project/cellbgnet/notebooks')

from skimage.io import imread
from skimage.measure import label
import random
import edt
from skimage.filters import gaussian
from scipy.ndimage import rotate
import random
import pickle
import pathlib
from pathlib import Path
%matplotlib qt5

In [3]:
param_file = '../cellbgnet/utils/reference_files/reference.yaml'
param = cellbgnet.utils.param_io.ParamHandling().load_params(param_file)

In [4]:
psf_params = param.PSF.to_dict()
simulation_params = param.Simulation.to_dict()
hardware_params = param.Hardware.to_dict()
train_size = simulation_params['train_size']
calib_file = psf_params['calib_file']

In [5]:
psf_size = 41
xextent = [-0.5, 127.5]
yextent = [-0.5, 127.5]
zextent = [-400, 400]
scale = torch.tensor([(xextent[1] - xextent[0]),
                      (yextent[1] - yextent[0]),
                      (zextent[1] - zextent[0])])
shift = torch.tensor([xextent[0], yextent[0], zextent[0]])

In [6]:
psf = SMAPSplineCoefficient(calib_file=calib_file).init_spline(
xextent=[-0.5, 127.5], yextent=[-0.5, 127.5], img_shape=[128, 128], device='cpu', 
roi_size=None, roi_auto_center=None)

INITIATING CPU IMPLEMENTATION


In [7]:
xyz = (torch.tensor([60, 60, 0])).view(-1, 3)
photon_scale = 1000
photons = torch.ones((xyz.shape[0]))*photon_scale

In [9]:
im = psf.forward(xyz, photons);


In [10]:
plt.imshow(im[0, :,:])

In [11]:
photon_sum = []

for z in range(-500, 500):
    xyz = (torch.tensor([60, 60, z])).view(-1, 3)
    photon_scale = 1000
    photons = torch.ones((xyz.shape[0]))*photon_scale
    im = psf.forward(xyz, photons);
    photon_sum.append(im.sum())




tensor(999.9989)